In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
from sqlalchemy import create_engine




In [32]:
# Get Suicide data  
Suicide_data = pd.DataFrame(pd.read_csv("resources/master.csv.zip")) 
Suicide_data.head(-5)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...,...,...
27810,Uzbekistan,2014,female,15-24 years,347,2992817,11.59,Uzbekistan2014,0.675,"63,067,077,179",2309,Millenials
27811,Uzbekistan,2014,male,55-74 years,144,1271111,11.33,Uzbekistan2014,0.675,"63,067,077,179",2309,Boomers
27812,Uzbekistan,2014,male,15-24 years,347,3126905,11.10,Uzbekistan2014,0.675,"63,067,077,179",2309,Millenials
27813,Uzbekistan,2014,male,75+ years,17,224995,7.56,Uzbekistan2014,0.675,"63,067,077,179",2309,Silent


In [17]:
Suicide_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             27820 non-null  object 
 1   year                27820 non-null  int64  
 2   sex                 27820 non-null  object 
 3   age                 27820 non-null  object 
 4   suicides_no         27820 non-null  int64  
 5   population          27820 non-null  int64  
 6   suicides/100k pop   27820 non-null  float64
 7   country-year        27820 non-null  object 
 8   HDI for year        8364 non-null   float64
 9    gdp_for_year ($)   27820 non-null  object 
 10  gdp_per_capita ($)  27820 non-null  int64  
 11  generation          27820 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.5+ MB


In [83]:
# Add data series to hold year 2015


suicide_2015 = Suicide_data.query("year == 2015")  
suicide_2015.head(-5)
suicide_2015 = suicide_2015.rename(columns={"country": "Country"})
suicide_2015.head()
#suicides_num_country = suicide_2015("Country")["suicides_no"].sum()
#suicides_num_country.head()
suicides_num_country = suicide_2015.groupby(["Country"])["suicides_no"].sum()
suicides_num_country

Country
Antigua and Barbuda        1
Argentina               3073
Armenia                   74
Australia               3027
Austria                 1251
                       ...  
Turkmenistan             133
Ukraine                 7574
United Kingdom          4910
United States          44189
Uruguay                  630
Name: suicides_no, Length: 62, dtype: int64

In [84]:
suicides_num_country_population = suicide_2015.groupby(["Country"])["population"].sum()
suicides_num_country_population

Country
Antigua and Barbuda        91889
Argentina               39699624
Armenia                  2795335
Australia               22240785
Austria                  8219386
                         ...    
Turkmenistan             4886514
Ukraine                 40345446
United Kingdom          61082942
United States          300078511
Uruguay                  3190795
Name: population, Length: 62, dtype: int64

In [89]:
suicides_agg=pd.merge(suicides_num_country_population,suicides_num_country, on='Country')
suicides_agg.head()

,population,suicides_no
Country,,
Antigua and Barbuda,91889,1
Argentina,39699624,3073
Armenia,2795335,74
Australia,22240785,3027
Austria,8219386,1251


In [91]:
#calulate suicide rate per 100 thousand people
suicides_agg['avg_suicides']=(suicides_agg['suicides_no']/suicides_agg['population'])*100000
suicides_agg

,population,suicides_no,avg_suicides
Country,,,
Antigua and Barbuda,91889,1,1.088270
Argentina,39699624,3073,7.740627
Armenia,2795335,74,2.647268
Australia,22240785,3027,13.610131
Austria,8219386,1251,15.220115
...,...,...,...
Turkmenistan,4886514,133,2.721777
Ukraine,40345446,7574,18.772875
United Kingdom,61082942,4910,8.038251


In [51]:
# suicide_2016 = Suicide_data.query("year == 2016")  
# suicide_2016.head(-5)
suicide_2015 = Suicide_data.query("year == 2015")  

# suicide_2016 = suicide_2016.rename(columns={"country": "Country"})
# suicide_2016.head()

,Country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
1248,Armenia,2016,male,75+ years,12,61956,19.37,Armenia2016,NaN,"10,546,135,160",3788,Silent
1249,Armenia,2016,male,55-74 years,16,237813,6.73,Armenia2016,NaN,"10,546,135,160",3788,Boomers
1250,Armenia,2016,male,35-54 years,16,350633,4.56,Armenia2016,NaN,"10,546,135,160",3788,Generation X
1251,Armenia,2016,male,15-24 years,5,202220,2.47,Armenia2016,NaN,"10,546,135,160",3788,Millenials
1252,Armenia,2016,female,75+ years,2,102414,1.95,Armenia2016,NaN,"10,546,135,160",3788,Silent


In [67]:
suicide_2015_distCountry = suicide_2015.groupby(suicide_2015["Country"]).nunique()
suicide_2015_distCountry

,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
Country,,,,,,,,,,,
Antigua and Barbuda,1,2,6,2,12,2,1,0,1,1,5
Argentina,1,2,6,12,12,12,1,0,1,1,5
Armenia,1,2,6,9,12,11,1,0,1,1,5
Australia,1,2,6,12,12,12,1,0,1,1,5
Austria,1,2,6,11,12,12,1,0,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...
Turkmenistan,1,2,6,10,12,10,1,0,1,1,5
Ukraine,1,2,6,12,12,12,1,0,1,1,5
United Kingdom,1,2,6,12,12,12,1,0,1,1,5


In [10]:
# Get happiness data 2015 
happiness_data_2015 = pd.DataFrame(pd.read_csv("resources/2015.csv")) 
happiness_data_2015.head(5)

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [61]:
happines_2015_distCountry = happiness_data_2015["Country"].value_counts()
happines_2015_distCountry

Switzerland    1
Bangladesh     1
Greece         1
Lebanon        1
Hungary        1
              ..
Kazakhstan     1
Slovenia       1
Lithuania      1
Nicaragua      1
Togo           1
Name: Country, Length: 158, dtype: int64

In [57]:
# # Get happiness data  2016
# happiness_data_2016 = pd.DataFrame(pd.read_csv("resources/2016.csv")) 
# happiness_data_2016.head(5)

,Country,Region,Happiness Rank,Happiness Score,Lower Confidence Interval,Upper Confidence Interval,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Denmark,Western Europe,1,7.526,7.460,7.592,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939
1,Switzerland,Western Europe,2,7.509,7.428,7.590,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,Iceland,Western Europe,3,7.501,7.333,7.669,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137
3,Norway,Western Europe,4,7.498,7.421,7.575,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465
4,Finland,Western Europe,5,7.413,7.351,7.475,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596


In [74]:
happy_suicide_2015 = pd.merge( suicide_2015, happiness_data_2015, on = 'Country')
happy_suicide_2015.head(5)


,Country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),...,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Argentina,2015,male,75+ years,245,712691,34.38,Argentina2015,NaN,"594,749,285,413",...,30,6.574,0.04612,1.05351,1.24823,0.78723,0.44974,0.08484,0.11451,2.836
1,Argentina,2015,male,15-24 years,656,3500580,18.74,Argentina2015,NaN,"594,749,285,413",...,30,6.574,0.04612,1.05351,1.24823,0.78723,0.44974,0.08484,0.11451,2.836
2,Argentina,2015,male,55-74 years,431,3011963,14.31,Argentina2015,NaN,"594,749,285,413",...,30,6.574,0.04612,1.05351,1.24823,0.78723,0.44974,0.08484,0.11451,2.836
3,Argentina,2015,male,25-34 years,465,3289927,14.13,Argentina2015,NaN,"594,749,285,413",...,30,6.574,0.04612,1.05351,1.24823,0.78723,0.44974,0.08484,0.11451,2.836
4,Argentina,2015,male,35-54 years,633,5166179,12.25,Argentina2015,NaN,"594,749,285,413",...,30,6.574,0.04612,1.05351,1.24823,0.78723,0.44974,0.08484,0.11451,2.836


In [73]:
# happy_suicide_nulls_series = pd.isnull(happy_suicide_2015["Happiness Score"])
# happy_suicide_2015[happy_suicide_nulls_series]

,Country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),...,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
